In [ ]:
#import packages
!pip install praw
import requests
import pandas as pd
import praw

In [2]:
#my details
reddit = praw.Reddit(client_id='X1xh8N9D99a34s9BoRYCFA', client_secret='0KvruvvglL3-iUbf6T6Vtv9zyuWZFQ', user_agent='Scrapping - 301')

In [ ]:
#getting reddit post
post_url = 'https://www.reddit.com/r/technology/comments/123nfrx/theres_a_90_chance_tiktok_will_be_banned_in_the/'
post = reddit.submission(url=post_url)
comments = post.comments.list()


In [6]:
#seeing the number of comments
post.num_comments

5527

In [ ]:
# @title
#package for comments within post
from praw.models import MoreComments

#creating empty list for post comments
post_comments = []

#collecting the body of each comment in the comment forest
for comment in post.comments:
	if type(comment) == MoreComments:
		continue

#appends the comment body
	post_comments.append(comment.body)

#create dataframe and putting all comments under "comments" column
all_comments_df = pd.DataFrame(post_comments, columns=['comments'])
all_comments_df


In [ ]:
#including all the comments within comments
all_comments = []

#including all the comments within "load more comments"
post.comments.replace_more(limit=None)
#replace the MoreComments objects with actual Comment objects
#iterates over all the comments in the comment forest
for comment in post.comments.list():
 all_comments.append([comment.body, comment.id, comment.score, comment.created])
# creating a dataframe
all_comments_df = pd.DataFrame(all_comments, columns=['comment', 'ID', 'score', 'created'])
all_comments_df


In [ ]:
#looking at our dataframe
all_comments_df.shape

In [ ]:
#converting the created date

from datetime import datetime
#all time details obtained
all_comments_df['date_time'] = pd.to_datetime(all_comments_df['created'],  unit='s')
#create a string with only the date
all_comments_df['date'] = all_comments_df['date_time'].dt.strftime('%Y-%m-%d')

In [ ]:
#looking at our dataframe
all_comments_df.shape

In [ ]:
#drop the duplicates
cleaned_comments_df = all_comments_df
cleaned_comments_df.drop_duplicates(subset='comment', keep='last', inplace=True)
cleaned_comments_df.to_csv('post_comments.csv', index=True, header=True)
#compare the new data after dropping duplicates
cleaned_comments_df.shape

In [ ]:
#view columns
cleaned_comments_df.columns

In [ ]:
#install package for text cleaning
!pip install spacy
import spacy
#pretrained model containing word features and more
spacy.cli.download("en_core_web_sm")
#process text and extract linguistic features
nlp = spacy.load("en_core_web_sm")


In [ ]:
#I will use re.sub for substituting string with a replacement string
import re
#remove the stopwords from text for NLP
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [ ]:
#function designed to clean the text
def reddit_clean(redd):
    redd = str(redd).lower()
    redd = re.sub("'", "", redd)
    redd = re.sub("@[A-Za-z0-9_]+","", redd)
    redd = re.sub("#[A-Za-z0-9_]+","", redd)
    redd = re.sub(r"www.\S+", "", redd)
    redd = re.sub(r"http\S+", "", redd)
    redd = re.sub('[()!?]', ' ', redd)
    redd = re.sub('\[.*?\]',' ', redd)
    redd = re.sub("[^a-z0-9]"," ", redd)
    redd = re.sub(r"\b\w{1,3}\b"," ", redd)
    redd = redd.split()
    #nltk stopwords to be removed
    stopwords = STOPWORDS
    redd = [w for w in redd if not w in stopwords]
    doc = nlp(" ".join(redd))
    #using spacy to remove parts of speech that are not needed
    redd = [token.text for token in doc if token.pos_ not in ['PRON', 'ADV','SCONJ', 'DET', 'AUX','ADP', 'PART']]
    redd = " ".join(word for word in redd)
    return redd

cleaned_comments_df['comment'] = cleaned_comments_df['comment'].apply(reddit_clean)
cleaned_comments_df.head()

In [ ]:
#tokenize the comments
tokenized_reddit_post = cleaned_comments_df['comment'].apply(lambda x: x.split())
print(tokenized_reddit_post)

In [ ]:
#stem the words

from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

tokenized_reddit_post = tokenized_reddit_post.apply(lambda x: [stemmer.stem(i) for i in x])
cleaned_comments_df['tokenized']= tokenized_reddit_post
cleaned_comments_df.head()

In [ ]:
#importing necessary packages for visualization
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import collections
#form a single string of all the words in the comments
all_words = ' '.join([text for text in cleaned_comments_df['comment']])
#dictionary where each key is a word and the corresponding value is the frequency of that word in the comments
count_word = collections.Counter(all_words.split())

#creating word cloud
wordcloud = WordCloud(width=800, height=500)
wordcloud.generate_from_frequencies(count_word)
plt.figure(figsize=(10, 7)) # In inches
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
import itertools
import collections
#count the most words shown up in all_words
count_word = collections.Counter(all_words.split())
count_word.most_common(15)

In [ ]:
#bar chart for top 10 frequent words
import seaborn as sns
a = nltk.FreqDist(count_word)
d = pd.DataFrame({'Reddit': list(a.keys()),
                  'Count': list(a.values())})
#selecting top 10 most frequent words
d = d.nlargest(columns="Count", n = 10)
plt.figure(figsize=(16,5))
ax = sns.barplot(data=d, x= "Reddit", y = "Count")
ax.set(ylabel = 'Word Count')
plt.show()

In [ ]:
#using vader for sentiment analysis
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#initialize VADER analyzer
analyzer = SentimentIntensityAnalyzer()

#create new columns for sentiment scores
cleaned_comments_df['neg'] = cleaned_comments_df['comment'].apply(lambda x: analyzer.polarity_scores(x)['neg'])
cleaned_comments_df['neu'] = cleaned_comments_df['comment'].apply(lambda x: analyzer.polarity_scores(x)['neu'])
cleaned_comments_df['pos'] = cleaned_comments_df['comment'].apply(lambda x: analyzer.polarity_scores(x)['pos'])
cleaned_comments_df['compound'] = cleaned_comments_df['comment'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

cleaned_comments_df.head()

In [ ]:
#using texblob for sentiment analysis (subjectivity)
from textblob import TextBlob

#obtaining subjectivity of the words
cleaned_comments_df['subjectivity'] = cleaned_comments_df['comment'].apply(lambda y: TextBlob(y).sentiment.subjectivity)
cleaned_comments_df.head()

In [ ]:
#create a function to classify the sentiment using vader
def analyze_sentiment(text):
    sentiment_scores = analyzer.polarity_scores(text)
    compound_score = sentiment_scores['compound']
    if compound_score >= 0.05:
        return 'Positive Sentiment'
    elif compound_score <= -0.05:
        return 'Negative Sentiment'
    else:
        return 'Neutral Sentiment'

In [ ]:
import numpy as np

In [ ]:
#create new column for sentiment and polarity using vader
cleaned_comments_df['sentiment'] = cleaned_comments_df['comment'].apply(lambda x: analyze_sentiment(x))

cleaned_comments_df['polarity'] = cleaned_comments_df['comment'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

#create a new subjectivity column
cleaned_comments_df['subjectivity2'] = np.where(cleaned_comments_df['subjectivity'] > 0.5, 'Subjective', 'Objective')
cleaned_comments_df.head()

In [ ]:
import pandas as pd
from collections import Counter

#select only the rows with Negative Sentiment
neg_comments = cleaned_comments_df[cleaned_comments_df['sentiment'] == 'Negative Sentiment']

#join all the comments into a single string
neg_text = ' '.join(neg_comments['comment'].tolist())

# split the string into words
neg_words = neg_text.split()

#count the frequency of each word
neg_word_counts = Counter(neg_words)

#get the top 10 most frequent words
top_neg_words = neg_word_counts.most_common(10)
words = [word[0] for word in top_neg_words]
counts = [count[1] for count in top_neg_words]

#creating bar chart
plt.bar(words, counts)
plt.xticks(rotation=45)
plt.xlabel('Word')
plt.ylabel('Count')
plt.title('Use of Top 10 Words within Negative Sentiment Context')
plt.show()


In [ ]:
#select only the rows with Negative Sentiment
pos_comments = cleaned_comments_df[cleaned_comments_df['sentiment'] == 'Positive Sentiment']

#join all the comments into a single string
pos_text = ' '.join(pos_comments['comment'].tolist())

#split the string into words
pos_words = pos_text.split()

#count the frequency of each word
pos_word_counts = Counter(pos_words)

#get the top 10 most frequent words
top_pos_words = pos_word_counts.most_common(10)
words = [word[0] for word in top_pos_words]
counts = [count[1] for count in top_pos_words]

#barchart
plt.bar(words, counts)
plt.xticks(rotation=45)
plt.xlabel('Word')
plt.ylabel('Count')
plt.title('Use of Top 10 Words within Positive Sentiment Context')
plt.show()

In [ ]:
#select only the rows with Negative Sentiment
neu_comments = cleaned_comments_df[cleaned_comments_df['sentiment'] == 'Neutral Sentiment']

#join all the comments into a single string
neu_text = ' '.join(neu_comments['comment'].tolist())

#split the string into words
neu_words = neu_text.split()

#count the frequency of each word
neu_word_counts = Counter(neu_words)

#get the top 10 most frequent words
top_neu_words = neu_word_counts.most_common(10)
words = [word[0] for word in top_neu_words]
counts = [count[1] for count in top_neu_words]

#barchart
plt.bar(words, counts)
plt.xticks(rotation=45)
plt.xlabel('Word')
plt.ylabel('Count')
plt.title('Use of Top 10 Words within Neutral Sentiment Context')
plt.show()

In [ ]:
#bar chart of the sentiment
plt.figure(figsize=(10,5))
plt.xlabel('Sentiment', fontsize=30)
plt.xticks(fontsize=15)
plt.ylabel('Frequency', fontsize=30)
plt.yticks(fontsize=15)
plt.hist(cleaned_comments_df['polarity'], bins=25)
plt.title('Sentiment Distribution', fontsize=30)
plt.show()

In [ ]:
#bar chart for subjectivity
plt.figure(figsize=(10,5))
plt.xlabel('Subjectivity', fontsize=30)
plt.xticks(fontsize=15)
plt.ylabel('Frequency', fontsize=30)
plt.yticks(fontsize=15)
plt.hist(cleaned_comments_df['subjectivity'], bins=15)
plt.title('Subjectivity Distribution', fontsize=30)
plt.show()

In [ ]:
#descriptive statistics for polarity and subjectivity
pol_sub = cleaned_comments_df[["polarity", "subjectivity"]]
pol_sub.describe()

In [ ]:
#table for the subjectivity of comments
pd.crosstab(cleaned_comments_df['sentiment'], cleaned_comments_df['subjectivity2'])